## Data preparation

In [25]:
import numpy as np
import scipy
import os
import sklearn as skl
import pandas as pd
import sklearn.utils, sklearn.preprocessing, sklearn.decomposition, sklearn.svm
import pylab as plt
from pandas.plotting import scatter_matrix
import librosa  #need install first
#import librosa.display

In [11]:
# Load metadata and features.
tracks = pd.read_csv('/Users/rulanxiao/Desktop/fma_metadata/tracks.csv', header=None)
echonest = pd.read_csv('/Users/rulanxiao/Desktop/fma_metadata/echonest.csv', header=None)

/Users/rulanxiao/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,1,5,6,8,12,18,20,21,22,24,33,34,38,39,44,47,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/rulanxiao/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,11,13,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175

In [12]:
# Change column names from "track.1", ... to named columns
header = tracks.iloc[1]
header[0] = 'track_ID'
tracks.drop(tracks.index[[0,1,2]], inplace=True)
tracks.rename(columns=header, inplace=True)
tracks.head()

header = echonest.iloc[2]
header[0]='track_ID'
echonest.drop(echonest.index[[0,1,2,3]],inplace=True)
echonest.rename(columns = header,inplace=True)
echonest.head()

# Only select songs for which we have echonest data
tracks_with_echonest_data = tracks[tracks['track_ID'].isin(echonest['track_ID'])]
echonest_data = echonest[echonest['track_ID'].isin(tracks_with_echonest_data['track_ID'])]

merged_echonest_data = pd.merge(tracks_with_echonest_data, echonest_data, on = 'track_ID')

# Change duplicate "listens" column to track listens and album listens
duplicate_listens = {'listens': ['album_listens', 'track_listens']}

merged_echonest_data = merged_echonest_data.rename(columns=lambda c: duplicate_listens[c].pop(0) if c in duplicate_listens.keys() else c)

In [13]:
# Extract year released from release date variable
album_release_year = []

for x in merged_echonest_data['date_released']:
    if type(x)==str:
        album_release_year.append(x[0:4])
    else: 
        album_release_year.append(np.nan)

#add album release year to dataframe
merged_echonest_data['album_release_year'] = album_release_year

# Check number of songs released per year and pick year with maximum
import collections
counter=collections.Counter(album_release_year)
print(counter)
print(counter.values())
print(counter.keys())
print(counter.most_common(3))

Counter({nan: 3140, '2010': 1461, '2009': 1426, '2011': 1101, '2008': 1088, '2007': 464, '2012': 380, '2006': 346, '2013': 298, '2014': 245, '2004': 167, '2005': 150, '2015': 145, '2003': 94, '2002': 84, '2001': 69, '1999': 68, '2000': 34, '1997': 25, '1995': 20, '1982': 19, '1998': 17, '1996': 10, '1986': 10, '1981': 6, '1973': 5, '1985': 5, '1992': 3, '1913': 2, '1976': 2, '1912': 1, '1916': 1, '1911': 1, '1909': 1, '1905': 1, '1907': 1, '1988': 1, '1902': 1})
dict_values([1426, 1088, 464, 150, 20, 346, 3140, 94, 19, 6, 69, 84, 167, 34, 68, 10, 1, 1, 1, 2, 1, 1, 1, 25, 3, 1, 17, 1461, 2, 10, 1101, 5, 245, 380, 298, 5, 1, 145])
dict_keys(['2009', '2008', '2007', '2005', '1995', '2006', nan, '2003', '1982', '1981', '2001', '2002', '2004', '2000', '1999', '1996', '1912', '1916', '1911', '1913', '1909', '1905', '1907', '1997', '1992', '1988', '1998', '2010', '1976', '1986', '2011', '1973', '2014', '2012', '2013', '1985', '1902', '2015'])
[(nan, 3140), ('2010', 1461), ('2009', 1426)]


In [14]:
# For chosen year, check the number of songs released per genre; pick top three genres to use
tracks_2010 = merged_echonest_data[merged_echonest_data['album_release_year'] == '2010']

# Top genres for 2010
counter=collections.Counter(tracks_2010['genre_top'])
print(counter)
print(counter.values())
print(counter.keys())
print(counter.most_common(10))

# Remove NAs, if needed 

# Select sensical variables from merged dataset (i.e., some track metadata and named echonest features)

# Stuff above - Kathy

Counter({nan: 524, 'Rock': 372, 'Electronic': 271, 'Hip-Hop': 133, 'Folk': 67, 'Pop': 58, 'Jazz': 23, 'International': 12, 'Instrumental': 1})
dict_values([23, 372, 58, 271, 524, 133, 67, 12, 1])
dict_keys(['Jazz', 'Rock', 'Pop', 'Electronic', nan, 'Hip-Hop', 'Folk', 'International', 'Instrumental'])
[(nan, 524), ('Rock', 372), ('Electronic', 271), ('Hip-Hop', 133), ('Folk', 67), ('Pop', 58), ('Jazz', 23), ('International', 12), ('Instrumental', 1)]


In [15]:
# Add sentiment analysis by title into dataset - compound score of positive / negative sentiment for song title
# Rulan
# require to install nltk first
# this part work nice, file will be generated to our project folder. 

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

tracks_senti = pd.read_csv('/Users/rulanxiao/Desktop/fma_metadata/tracks.csv',header=None)

header = tracks_senti.iloc[1]
header[0]='track ID'
header[52]='track title'
tracks_senti.drop(tracks_senti.index[[0,1,2]],inplace=True)
tracks_senti.rename(columns = header,inplace=True)
tracks_senti.head()

df=tracks_senti[['track ID','track title']]
df.dropna(axis=0, how='any')

ml = df["track title"].values
title=[]
for i in range(len(ml)):
    a=str(ml[i])
    title.append(a)
idd = df["track ID"].values

neg=[]
neu=[]
pos=[]
comp=[]

sid = SentimentIntensityAnalyzer()
for sentence in title:
    ss = sid.polarity_scores(sentence)
    score=[]
    for k in ss:
        a=ss[k]
        score.append(a)
    neg.append(score[0])
    neu.append(score[1])
    pos.append(score[2])
    comp.append(score[3])

sentimentall = pd.DataFrame({'track_ID':idd,'track_title':title,'senti neg': neg,'senti neu': neu,'senti pos': pos,'senti comp': comp})

sentimentall.to_csv('sentimental_analysis_title.csv')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/rulanxiao/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


/Users/rulanxiao/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,1,5,6,8,12,18,20,21,22,24,33,34,38,39,44,47,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
sentiment= pd.read_csv('sentimental_analysis_title.csv', header=[0])
#sentiment.rename(index=str, columns={'track_ID' : 'track ID'},inplace=True)
sentiment=sentiment[['track_ID','senti comp', 'senti neg','senti pos']]
tracks_2010 = pd.merge(tracks_2010, sentiment, how='inner', on=['track_ID'])

In [17]:
# For baseline model - add genre as a categorical variable 
# Sa

t10 = tracks_2010[['track_ID','latitude','longitude','bit_rate','duration','genre_top','genres_all','track_listens','acousticness','danceability','energy','instrumentalness','liveness','speechiness','tempo','valence','artist_hotttnesss','senti comp', 'senti neg','senti pos']]
t10['Rock'] = (t10['genre_top'] == 'Rock').astype(int)
t10['Hip-Hop'] = (t10['genre_top'] == 'Hip-Hop').astype(int)
t10['Electronic'] = (t10['genre_top'] == 'Electronic').astype(int)

# For genre-specific models - split dataset into three based on top three genres
# Sa

t10.groupby(['genre_top']).size()
Hiphop_10=t10.loc[t10['genre_top'] == 'Hip-Hop']
Rock_10=t10.loc[t10['genre_top'] == 'Rock']
Elec_10=t10.loc[t10['genre_top'] == 'Electronic']





In [18]:
# Make boxplots of variables in genre dataset 
# Sa
t10.boxplot()
# Standardize variables as needed
# Sa
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
y = np.array(t10[['track_listens']].values)
X=(t10[['bit_rate','duration','track_listens','acousticness','danceability','energy','instrumentalness','liveness','speechiness','tempo','valence','artist_hotttnesss','senti comp', 'senti neg','senti pos']].values)
scaler = preprocessing.StandardScaler()
X_scaled = scaler.fit_transform(X)
y_scaled = scaler.fit_transform(y.reshape(-1, 1))
# Repeat for other two genres
# Sa

ValueError: Found array with 0 sample(s) (shape=(0, 15)) while a minimum of 1 is required by StandardScaler.

## Exploratory analysis

In [23]:
# Check for correlation between variables with correlation plot 
# Kathy

# Check for correlation between arist hotness, familiarity, discovery
# Kathy
t10

,track_ID,latitude,longitude,bit_rate,duration,genre_top,genres_all,track_listens,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence,artist_hotttnesss,senti comp,senti neg,senti pos


In [31]:
# Make scatter plots of numerical variables versus outcome (track listens)
# Rulan

plt.scatter(tracks.track_listens,tracks.acousticness)
plt.ylabel('track_listens')
plt.xlabel('duration')
plt.title('track_listens v.s. duration')
plt.show()

plt.scatter(t10.track_listens,t10.acousticness)
plt.ylabel('track_listens')
plt.xlabel('acousticness')
plt.title('track_listens v.s. acousticness')
plt.show()

plt.scatter(t10.track_listens,t10.acousticness)
plt.ylabel('track_listens')
plt.xlabel('energy')
plt.title('track_listens v.s. energy')
plt.show()

plt.scatter(t10.track_listens,t10.acousticness)
plt.ylabel('track_listens')
plt.xlabel('instrumentalness')
plt.title('track_listens v.s. instrumentalness')
plt.show()

plt.scatter(t10.track_listens,t10.acousticness)
plt.ylabel('track_listens')
plt.xlabel('liveness')
plt.title('track_listens v.s. liveness')
plt.show()

plt.scatter(t10.track_listens,t10.acousticness)
plt.ylabel('track_listens')
plt.xlabel('speechiness')
plt.title('track_listens v.s. speechiness')
plt.show()

plt.scatter(t10.track_listens,t10.acousticness)
plt.ylabel('track_listens')
plt.xlabel('tempo')
plt.title('track_listens v.s. tempo')
plt.show()

scatter_matrix(??,figsize=(30,30))
plt.show()

# Correlation plots for categorical variables

SyntaxError: invalid syntax (<ipython-input-31-f89499530198>, line 46)

## Model selection

In [ ]:
# Baseline model - use the year with most songs data and include genre as a categorical variable (use the same top 3 genres)

# Convert genre to dummy variables

# Use linear regression with same variables as above (sentiment analysis, track metadata, echonest named variable, location)

# IDEA: we will see that genre is a big predictor of track listens - thus, we train separate models for each genre to dig deeper into why that is

In [ ]:
# For 5-folds, split data into training and testing (one of the genres) - will repeat for other two genres (maybe write function to do this)

# Train linear regression model using all variables

# Regularize using L^1 penalty due to large number of features - pick optimal penalty and compare R^2

# Use GridSearchCV with at least 2-fold validation

# Check resulting model on 5 random folds of data 

# Plot coefficients for resulting model 

In [ ]:
# Train random forest regression model 

# Regularize using L^1 penalty due to large number of features - pick optimal penalty and compare R^2

# Use GridSearchCV with at least 2-fold validation

# Check resulting model on 5 random folds of data 

# Plot coefficients for resulting model 

In [ ]:
# Explore adding other features to model, such as nonlinear features (interaction terms)

In [ ]:
# Rock

In [ ]:
# Jazz

In [ ]:
# Hip hop

In [ ]:
# Compare how models are different between different genres - for instance, if certain variables are more or less important for different genres

In [ ]:
# Write function that takes in optimal_model, new song audio and necessary info (like genre, track title, etc.) and uses librosa to extract features
# and output a prediction of number of track listens 

# Load the example clip
# Load 30 seconds of a wav file, starting 15 seconds in
y, sr = librosa.load('blahblah.mp3', offset=15.0, duration=30.0)

# Set the hop length; at 22050 Hz, 512 samples ~= 23ms
hop_length = 512

# Separate harmonics and percussives into two waveforms
y_harmonic, y_percussive = librosa.effects.hpss(y)

# Beat track on the percussive signal
tempo, beat_frames = librosa.beat.beat_track(y=y_percussive,
                                             sr=sr)

# Compute MFCC features from the raw signal
mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=13)

# And the first-order differences (delta features)
mfcc_delta = librosa.feature.delta(mfcc)

# Stack and synchronize between beat events
# This time, we'll use the mean value (default) instead of median
beat_mfcc_delta = librosa.util.sync(np.vstack([mfcc, mfcc_delta]),
                                    beat_frames)

# Compute chroma features from the harmonic signal
chromagram = librosa.feature.chroma_cqt(y=y_harmonic,
                                        sr=sr)

# Aggregate chroma features between beat events
# We'll use the median value of each feature between beat frames
beat_chroma = librosa.util.sync(chromagram,
                                beat_frames,
                                aggregate=np.median)

# Finally, stack all beat-synchronous features together
beat_features = np.vstack([beat_chroma, beat_mfcc_delta])

In [ ]:
##Statistic for each feature
def columns():
    feature_sizes = dict(chroma_stft=12, chroma_cqt=12, chroma_cens=12,
                         tonnetz=6, mfcc=20, rmse=1, zcr=1,
                         spectral_centroid=1, spectral_bandwidth=1,
                         spectral_contrast=7, spectral_rolloff=1)
    moments = ('mean', 'std', 'skew', 'kurtosis', 'median', 'min', 'max')

    columns = []
    for name, size in feature_sizes.items():
        for moment in moments:
            it = ((name, moment, '{:02d}'.format(i+1)) for i in range(size))
            columns.extend(it)

    names = ('feature', 'statistics', 'number')
    columns = pd.MultiIndex.from_tuples(columns, names=names)

    # More efficient to slice if indexes are sorted.
    return columns.sort_values()


def feature_stats(name, values):
    features[name, 'mean'] = np.mean(values, axis=1)
    features[name, 'std'] = np.std(values, axis=1)
    features[name, 'skew'] = stats.skew(values, axis=1)
    features[name, 'kurtosis'] = stats.kurtosis(values, axis=1)
    features[name, 'median'] = np.median(values, axis=1)
    features[name, 'min'] = np.min(values, axis=1)
    features[name, 'max'] = np.max(values, axis=1)


features = pd.Series(index=columns(), dtype=np.float32)


In [ ]:
##OTHER FEATURES
y, sr = librosa.load('Rough & Laugh.mp3', offset=15.0, duration=30.0)

stft=np.abs(librosa.stft(y, n_fft=2048, hop_length=512))
f = librosa.feature.chroma_stft(S=stft**2, n_chroma=12)
feature_stats('chroma_stft', f)

f = librosa.feature.zero_crossing_rate(y, frame_length=2048, hop_length=512)
feature_stats('zcr', f)

cqt = np.abs(librosa.cqt(y, sr=sr, hop_length=512, bins_per_octave=12,
                                 n_bins=7*12, tuning=None))
assert cqt.shape[0] == 7 * 12
assert np.ceil(len(y)/512) <= cqt.shape[1] <= np.ceil(len(y)/512)+1


f = librosa.feature.chroma_cqt(C=cqt, n_chroma=12, n_octaves=7)
feature_stats('chroma_cqt', f)

f = librosa.feature.chroma_cens(C=cqt, n_chroma=12, n_octaves=7)
feature_stats('chroma_cens', f)

f = librosa.feature.tonnetz(chroma=f)
feature_stats('tonnetz', f)

print(features)